In [195]:
import pandas as pd
from datetime import datetime as dt
import warnings
import os

path = os.path.join(os.getcwd()+"/Output")
 
if not (os.path.exists(path)):
    os.makedirs(path)
csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith(".csv"):
        csv_files.append(file)

for files in csv_files:
    df2 = pd.read_csv(files)
    df3 = pd.read_csv('Source/BANKNIFTY-I.csv')

    for col in df3.columns[2:]:
            df3[col].values[:] = 0  

    df3['Date'] = pd.to_datetime(df3['Date'],format="%Y/%m/%d")
    df3['Time'] = pd.to_datetime(df3['Time'],format="%H:%M:%S").dt.time

    df2['Date'] = pd.to_datetime(df2['Date'],format="%Y/%m/%d")
    df2['Time'] = pd.to_datetime(df2['Time'],format="%H:%M:%S").dt.time

    df3["DT"] = df3['Date'].astype(str) + ' ' + df3['Time'].astype(str)
 
    df2["DT"] = df2['Date'].astype(str) + ' ' + df2['Time'].astype(str)

    r1 = df2[(pd.to_datetime(df2['DT'])).isin(pd.to_datetime(df3['DT']))]

    r2 = df3[~(pd.to_datetime(df3['DT'])).isin(pd.to_datetime(df2['DT']))]

    df = pd.concat([r2,r1],axis=0,ignore_index = True)

    df = df.sort_values(by="DT",ignore_index = True)

    expiry = pd.read_csv("Source/Expiry(2015-22).csv")

    es = df['Date'].isin(expiry['Expiry'])

    df['Expiry']=0

    for i in range(len(df)-1):
        if ((es[i] == True) & ((str(df.loc[i+1,"Time"]) == '09:15:59') or (str(df.loc[i+1,"Time"]) == '9:15:59'))):
                df.loc[i+1,"Expiry"] = -1

    for i in range(len(df)-1):
        if ((df.loc[i+1,"Expiry"]==0) & (df.loc[i+1,"Open"]==0)): 
            df.loc[i+1,"Open"]=df.loc[i,"Close"]
            df.loc[i+1,'High'] = df.loc[i,'Close']
            df.loc[i+1,'Low'] = df.loc[i,'Close']
            df.loc[i+1,'Close'] = df.loc[i,'Close'] 
            df.loc[i+1,'Open Interest'] = df.loc[i,'Open Interest']
        

    df = df.drop(['DT', 'Expiry'], axis=1)
    df.to_csv(f"Output/{files}",index=False)